In [65]:
import findspark
findspark.init("/Users/rahmanidriss/bigdata/tools/spark")
#findspark.init()
#import pyspark
#import SparkSession
from pyspark.sql import Row, SparkSession
from pyspark import SparkConf
from pyspark.sql.types import  StructType,StructField,StringType,DoubleType, IntegerType,TimestampType, _parse_datatype_string, _parse_datatype_json_string
import pyspark.sql.functions as F

In [74]:
conf = (SparkConf().set("spark.driver.memory", "8g")
            .set("spark.sql.execution.arrow.pyspark.enabled", "true")
            .set("spark.sql.execution.arrow.pyspark.fallback.enabled", "false")
            )
spark = (
    SparkSession.builder
    #.master("spark://172.30.0.3:7077")
    #.config(("spark.driver.memory", "8g"))
    #.config("spark.cores.max", 12)
    #.config("spark.executor.cores", 2)
    #.config("spark.executor.memory", "2g")
    .config("spark.sql.adaptive.enabled",'false')
    #.config('spark.sql.adaptive.coalescePartitions.initialPartitionNum', 24)
    #.config('spark.sql.adaptive.coalescePartitions.parallelismFirst', 'false')
    #.config('spark.sql.files.minPartitionNum', 1)
    #.config('spark.sql.files.maxPartitionBytes', '500mb')
    # or .config(conf=conf)
    .appName('parquet_strategies')
    .getOrCreate()
)
spark

24/02/05 21:52:24 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [12]:
spark.stop()

In [30]:
# Define a schema for the data
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("gender", StringType(), True)
])

# Generate some sample data
data = [
    (1, "John", 30, "Male"),
    (2, "John", 30, "Female"),
    (3, "John", 30, "Male"),
    (4, "Jane", 35, "Female"),
    (5, "David", 25, "Male"),
    (6, "David", 25, "Female"),
    (7, "David", 25, "Female"),
    (8, "David", 25, "Male"),
    (9, "Emily", 40, "Female")
]
df = spark.createDataFrame(data, schema)



In [31]:
df.show()

+---+-----+---+------+
| id| name|age|gender|
+---+-----+---+------+
|  1| John| 30|  Male|
|  2| John| 30|Female|
|  3| John| 30|  Male|
|  4| Jane| 35|Female|
|  5|David| 25|  Male|
|  6|David| 25|Female|
|  7|David| 25|Female|
|  8|David| 25|  Male|
|  9|Emily| 40|Female|
+---+-----+---+------+



In [38]:
# Write the data to a Parquet file with partitioning and bucketing
(df.write
    .mode("overwrite")
    .format("parquet")
    .partitionBy("age","gender")
    #.bucketBy(2, "age")
    #.sortBy("name")
    .option("compression", "snappy")
    .parquet("file://///Users/rahmanidriss/Desktop/dataScience/spark_hadoop_cluster/data/test.parquet")
)

24/02/05 12:49:10 WARN MemoryManager: Total allocation exceeds 95,00% (906 992 014 bytes) of heap memory
Scaling row group sizes to 96,54% for 7 writers


In [56]:
df_parquet=(
    spark.read
    .format("parquet")
    .load("file://///Users/rahmanidriss/Desktop/dataScience/spark_hadoop_cluster/data/test.parquet")
)



In [59]:
df_parquet.select("age").filter(F.col("age")<23).explain()

== Physical Plan ==
*(1) Project [age#308]
+- *(1) ColumnarToRow
   +- FileScan parquet [age#308,gender#309] Batched: true, DataFilters: [], Format: Parquet, Location: InMemoryFileIndex(1 paths)[file:/Users/rahmanidriss/Desktop/dataScience/spark_hadoop_cluster/data..., PartitionFilters: [isnotnull(age#308), (age#308 < 23)], PushedFilters: [], ReadSchema: struct<>




'200'

24/02/06 19:42:13 WARN HeartbeatReceiver: Removing executor driver with no recent heartbeats: 237631 ms exceeds timeout 120000 ms
24/02/06 19:42:14 WARN SparkContext: Killing executors is not supported by current scheduler.
24/02/06 19:42:16 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.SparkThreadUtils$.awaitResult(SparkThreadUtils.scala:56)
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:310)
	at org.apache.spark.rpc.RpcTimeout.awaitResult(RpcTimeout.scala:75)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRefByURI(RpcEnv.scala:102)
	at org.apache.spark.rpc.RpcEnv.setupEndpointRef(RpcEnv.scala:110)
	at org.apache.spark.util.RpcUtils$.makeDriverRef(RpcUtils.scala:36)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.driverEndpoint$lzycompute(BlockManagerMasterEndpoint.scala:124)
	at org.apache.spark.storage.BlockManagerMasterEndpoint.org$apache$spark$storage$BlockManagerMasterEndpoint$$